In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in os.listdir(maindir + '\\testing_modules'):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)

In [2]:
%matplotlib qt
import skimage.io as io
import pyFAI
from skimage.restoration import rolling_ball
from lmfit.models import PolynomialModel, ExponentialModel, SplineModel
import matplotlib
from matplotlib.widgets import Button, Slider

In [3]:
scanid = 149248
filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
dtype = np.int16

img_map = io.imread(f'{base}{filedir}{map_filename}')#.astype(np.float32, copy=False)

med_img = np.median(img_map, axis=(0, 1)).astype(dtype)
max_map = np.max(img_map, axis=(2, 3)).astype(dtype)
min_map = np.min(img_map, axis=(2, 3)).astype(dtype)
sum_map = np.sum(img_map, axis=(2, 3)).astype(dtype)

proc_map = img_map - med_img
del(img_map)
max_img = np.max(proc_map, axis=(0, 1)).astype(dtype)
min_img = np.min(proc_map, axis=(0, 1)).astype(dtype)
ref_img = max_img - min_img

print(proc_map.shape)
print(proc_map.dtype)

(61, 61, 486, 768)
int32


In [4]:
elements_folder = f'output_tiff_scan2D_{scanid}_xs_sum8ch\\'

element_maps = {}
for element in os.listdir(f'{filedir}{elements_folder}'):
    el_im = io.imread(f'{filedir}{elements_folder}{element}')
    element_maps[str(element[:-5])] = el_im

In [23]:
key = 'detsum_Ni_K'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(proc_map, range(proc_map.shape[0]), range(proc_map.shape[1]), display_map=element_maps[key], display_title=key)

In [14]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=200)

im = ax.imshow(med_img)
fig.colorbar(im, ax=ax)

plt.show()

In [5]:
ai = pyFAI.load(f'{filedir}scan149256_dexela.poni')
ai.energy = 15
tth_resolution = 0.02 # Degrees
chi_resolution = 0.05 # Degrees

res = ai.integrate2d_ng(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])[0], 100, 100, unit='2th_deg')
tth, chi = res[1], res[2]
tth_num = int(np.abs(np.max(tth) - np.min(tth)) // tth_resolution)
chi_num = int(np.abs(np.max(chi) - np.min(chi)) // chi_resolution)
# The bounds of interpolation should be limited by the intrument resolution AND the original image size. Should not make up too much data

# Is there a better way I could do this?
res = ai.integrate2d_ng(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])[0], tth_num, chi_num, unit='2th_deg')
tth, chi = res[1], res[2]
extent = [tth[0], tth[-1], chi[0], chi[-1]]

# Calibrate the full map. Can I parallelize this??
calibrated_map = np.zeros((proc_map.shape[0] * proc_map.shape[1], chi_num, tth_num), dtype=(dtype))
for i, pixel in enumerate(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])):
    res, _, _ = ai.integrate2d_ng(pixel, tth_num, chi_num, unit='2th_deg')
    calibrated_map[i] = res
calibrated_map = calibrated_map.reshape(proc_map.shape[0], proc_map.shape[1], chi_num, tth_num)

In [11]:
key = 'detsum_Ni_K'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(calibrated_map, tth, chi, display_map=element_maps[key], display_title=key)

In [8]:
fig, ax = plt.subplots(2, 1, figsize=(10, 5), dpi=200, sharex=True)

max_img = np.max(proc_map, axis=(0, 1)).astype(dtype)
min_img = np.min(proc_map, axis=(0, 1)).astype(dtype)
ref_img = max_img - min_img

ref_img.astype(np.float16)[ref_img < 0] = 0

res = ai.integrate2d_ng(ref_img, 4096, 360, unit='2th_deg')
tth, chi = res[1], res[2]
xrd_int = res[0]

im = ax[0].imshow(res[0], extent=[tth[0], tth[-1], chi[0], chi[-1]], aspect='auto', norm=LogNorm(vmin=100))
fig.colorbar(im, ax=ax)
ax[0].set_ylabel('Azimuthal Angle, χ [°]')
ax[1].set_xlabel('Scattering Angle, 2θ [°]')
_, res = ai.integrate1d_ng(ref_img, 4096, unit='2th_deg') # I really do not like that the 1D and 2D outputs are flipped
xrd_int = res
ax[1].plot(tth, res)

plt.show()

In [9]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
hematite = Phase.fromCIF(f'{cif_dir}AMCSD\\Hematite_0017806.cif')
hematite.name = 'Hematite'

iron_alpha = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron-alpha_0011214.cif')
iron_alpha.name = 'Iron-ferrite'

iron_beta = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron-beta_0011215.cif')
iron_beta.name = 'Iron-beta'

iron_delta = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron-delta_0011216.cif')
iron_delta.name = 'Iron-delta'

iron = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron_0011146.cif')
iron.name = 'Iron-austenite'

cementite = Phase.fromCIF(f'{cif_dir}AMCSD\\Cementite_0013523.cif')
cementite.name = 'Cementite'

zincite = Phase.fromCIF(f'{cif_dir}AMCSD\\Zincite_0011555.cif')
zincite.name = 'Zincite'

In [10]:
size=200
bkg_spline = SplineModel(xknots=np.linspace(tth[0], tth[-1], 20))
params = bkg_spline.guess(median_filter(xrd_int, size=(size)), tth)
bkg_fit = bkg_spline.fit(median_filter(xrd_int, size=(size)), params, x=tth)
bkg = bkg_fit.best_fit

In [24]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

#bkg = rolling_ball(xrd_int, radius=50)
#bkg_spline = UnivariateSpline(tth, xrd_int)
#bkg_spline.set_smoothing_factor(6e7)
#bkg = bkg_spline(tth)

ax.plot(tth, xrd_int, label='exp')
ax.plot(tth, median_filter(xrd_int, size=(size)), label='median')
ax.plot(tth, bkg, label='bkg')
ax.legend()

plt.show()

In [11]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=200)
colors = matplotlib.color_sequences['tab10']

norm_xrd_int = rescale_array(xrd_int - bkg, upper=100, arr_min=0)

phases = [hematite, iron, iron_alpha, iron_beta, iron_delta, cementite, zincite]
[phase.get_hkl_reflections(tth_range=(tth[0], tth[-1]), en=ai.energy*1e3) for phase in phases if phase.reflections is None];

xrd_plot = ax.plot(tth, norm_xrd_int, label='Composite XRD', c='k', zorder=(len(phases) + 1))
fig.subplots_adjust(right=0.75)

def update_max(LineCollection, phase, slider):
    seg = np.asarray(LineCollection.get_segments())
    seg[:, 1, 1] = slider.val * phase.reflections['int'] / np.min(phase.reflections['int'])
    return seg

lines = [xrd_plot]
slider_lst = []
update_lst = []

def update_factory(index):
    def update(val):
        lines[index + 1].set_segments(update_max(lines[index + 1], phases[index], slider_lst[index]))
        fig.canvas.draw_idle()
    return update

slider_vpos = np.linspace(0.8, 0.1, len(phases))
for i, phase in enumerate(phases):
    phase_intensities = phase.reflections['int']
    phase_scale = np.max(norm_xrd_int) / np.max(phase_intensities)
    phase_plot = ax.vlines(phase.reflections['tth'], ymin=0, ymax=0, color=colors[i], lw=2)
    lines.append(phase_plot)

    axphase = fig.add_axes([0.8, slider_vpos[i], 0.1, 0.03])
    axphase.set_title(phase.name, fontsize=10)
    phase_amp = Slider(
            ax=axphase,
            label='',
            valmin=0,
            valmax=100,
            valinit=0,
            initcolor='none',
            color = colors[i],
            handle_style={'edgecolor' : colors[i], 'size' : 8}
            )
    
    slider_lst.append(phase_amp)
    update_lst.append(update_factory(i))
    slider_lst[i].on_changed(update_lst[i])

ax.set_ylim(0)
ax.set_xlabel('Scattering Angle, 2θ [°]')
ax.set_title('Phase Selector')
ax.legend(fontsize=10)
plt.show()

In [12]:
test_map = ImageMap.

ValueError: maximum supported dimension for an ndarray is 32, found 61